# Alternative 2 - On a given feature set

In [12]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from sklearn import metrics

# NEW METHOD TO RESET SEED TO 0:
def reset_seed():
    tf.random.set_seed(0)
    np.random.seed(0)
    
def classification_results(y, yhat):
    acc = metrics.accuracy_score(y, yhat)
    mcc = metrics.matthews_corrcoef(y, yhat)
    f1_weighted = metrics.f1_score(y, yhat, average="weighted")
    return acc, mcc, f1_weighted

def evaluate_per_pid(df, test_range, nnmodel, x_test, name):
    
    df_held_out_test = df[df["pid"].isin(test_range)][["pid", "time", "sleep_phase", "sleep"]].copy()
    df_held_out_test["yhat"] = nnmodel.predict(x_test).round()
    df_held_out_test.to_csv(f"results/{name}.csv.tar.gz", index=False)
        
    final_results = df_held_out_test.groupby(["pid"])[["sleep", "yhat"]].apply(lambda x:
                                                                          classification_results(x["sleep"].values, 
                                                                                                 x["yhat"].values))

    final_results = final_results.apply(pd.Series).rename(columns={0: "Accuracy", 1: "MCC", 2: "F1_weighted"})

    return final_results.agg(["mean", "std"]).round(3)

reset_seed()

In [13]:
df = pd.read_csv("./datasets/df_tsfresh_features.tar.gz")
df.head()

,time,act,sleep_phase,hr,pid,act__variance_larger_than_standard_deviation,act__has_duplicate_max,act__has_duplicate_min,act__has_duplicate,act__sum_values,...,hr__fourier_entropy__bins_2,hr__fourier_entropy__bins_3,hr__fourier_entropy__bins_5,hr__fourier_entropy__bins_10,hr__fourier_entropy__bins_100,hr__permutation_entropy__dimension_3__tau_1,hr__permutation_entropy__dimension_4__tau_1,hr__permutation_entropy__dimension_5__tau_1,hr__permutation_entropy__dimension_6__tau_1,hr__permutation_entropy__dimension_7__tau_1
0,1,2.0,0.0,71.0,0,0.0,0.0,0.0,0.0,2.0,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,NaN,NaN,NaN,NaN,NaN
1,2,0.0,0.0,76.0,0,0.0,0.0,0.0,0.0,3.0,...,0.693147,0.693147,0.693147,0.693147,0.693147,-0.000000,NaN,NaN,NaN,NaN
2,3,1.0,0.0,78.0,0,0.0,1.0,0.0,1.0,5.0,...,0.636514,0.636514,0.636514,0.636514,0.636514,0.693147,-0.000000,NaN,NaN,NaN
3,4,2.0,0.0,73.0,0,1.0,0.0,0.0,1.0,92.0,...,0.636514,0.636514,0.636514,0.636514,1.098612,1.098612,0.693147,-0.000000,NaN,NaN
4,5,87.0,0.0,80.0,0,1.0,0.0,1.0,1.0,92.0,...,0.693147,0.693147,0.693147,0.693147,1.386294,1.386294,1.098612,0.693147,-0.0,NaN


In [14]:
df["sleep_phase"].unique()
# 0 -> Wake
# 1 -> phase 1 (light sleep)
# 2 -> phase 2 (deep sleep N1)
# 3 -> phase 3 (deep sleep N2)
# 4 -> NREM    ()
# 5 -> REM

array([0., 1., 2., 5., 3., 4.])

In [15]:
df["sleep"] = df["sleep_phase"] > 0

### Model high-level details:

- Model input (2, S, 11): 
    - Option 1: With data allignment:
                       [
                          [hr_0, hr_1, hr_2   ....hr_N]
                          [act_0, act_1, act_2....act_N]
                       ]

    - Option 2: Without data allignment:
                       [
                           [hr_0, hr_1, hr_2   ....hr_N, act_0, act_1, act_2....act_N]
                       ]
                       

- Model output:
    - (S, 1) (bin sleep phase)
   


### Get XY from dataframe

In [22]:
def generate_XY(df, ycol="sleep", align_cols=True):

    # This could be used in several different ways
    # e.g., removing nan cols, inputing averages, etc
    df = df.fillna(0.0)
    df = df.replace([np.inf, -np.inf], 0)
    
    hr_cols = [k for k in df.keys() if k.startswith("hr_")]
    act_cols = [k for k in df.keys() if k.startswith("act_")]

    hr_cols = sorted(hr_cols)
    act_cols = sorted(act_cols)

    if align_cols:
        hr = df[hr_cols].values
        act = df[act_cols].values
        X = np.stack((act,hr))
        X = X.transpose(1,0,2)
    else:
        X = df[hr_cols + act_cols].values

    Y  = df[ycol].values.reshape(-1, 1)
        
    return X, Y


In [23]:
# generate_XY(df, align_cols=True)   #  X.shape is (102759, 2, 768)
# generate_XY(df, align_cols=False).shape  #  X.shape is (102759, 1536)

In [46]:
align_cols = True
df_XY = df.groupby("pid").apply(lambda x: generate_XY(x, align_cols=align_cols))
df_XY.head()

pid
0    ([[[ 4.          2.         -1.         -1.   ...
1    ([[[ 1.          1.         -1.         -1.   ...
2    ([[[ 8.73000000e+02  1.50000000e+01 -1.0000000...
3    ([[[ 1.          1.         -1.         -1.   ...
4    ([[[ 1.00000000e+00  1.00000000e+00 -1.0000000...
dtype: object

In [47]:
idx = 3
df_XY.iloc[idx][0].shape, df_XY.iloc[idx][1].shape

((625, 2, 768), (625, 1))

In [48]:
xs, ys = [], []
for row_id, (x, y) in df_XY.items():
    xs.append(x)
    ys.append(y)
    
xs = np.array(xs, dtype=object)
ys = np.array(ys, dtype=object)


In [49]:
# Now we can create a simple trainset from the dataset making sure that
# data from one subject is NOT at the same time in the training and in the test sets
subjects_train_idx = [0, 1, 2, 3, 4]    
np.vstack(xs[subjects_train_idx]).shape, np.vstack(ys[subjects_train_idx]).shape

((4778, 2, 768), (4778, 1))

In [50]:
subjects_train_idx = range(0, 40)
X_train = np.vstack(xs[subjects_train_idx])
Y_train = np.vstack(ys[subjects_train_idx])

subjects_val_idx = range(40, 50)
X_val = np.vstack(xs[subjects_val_idx])
Y_val = np.vstack(ys[subjects_val_idx])

subjects_test_idx = range(50, 100)
X_test = np.vstack(xs[subjects_test_idx])
Y_test = np.vstack(ys[subjects_test_idx])


### Evaluate a few models

#### Simple Model

In [51]:
def simple_dense_model(num_features, align_cols):
    
    model = tf.keras.models.Sequential()
    if align_cols:
        model.add(tf.keras.layers.Dense(32, input_shape=(2, num_features), activation='relu'))
    else:
        model.add(tf.keras.layers.Dense(32, input_shape=(num_features, ), activation='relu'))
    
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

simple_model = simple_dense_model(num_features = X_train.shape[-1], align_cols=align_cols)
  

In [55]:
reset_seed()

# Check what happens when toggling align_cols above
with tf.device('/cpu:0'):
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = simple_model.fit(X_train, Y_train,
                               validation_data=(X_val, Y_val), 
                               epochs=50, 
                               batch_size=8,
                               callbacks=[early_stop_callback])


Epoch 1/50
5145/5145 [==============================] - 3s 601us/step - loss: 0.4893 - accuracy: 0.7864 - val_loss: 0.4792 - val_accuracy: 0.7818
Epoch 2/50
5145/5145 [==============================] - 3s 579us/step - loss: 551595.5000 - accuracy: 0.7883 - val_loss: 0.4812 - val_accuracy: 0.7824
Epoch 3/50
5145/5145 [==============================] - 3s 587us/step - loss: 0.4769 - accuracy: 0.7893 - val_loss: 0.4803 - val_accuracy: 0.7827
Epoch 4/50
5145/5145 [==============================] - 3s 597us/step - loss: 321857248.0000 - accuracy: 0.7897 - val_loss: 0.4790 - val_accuracy: 0.7824
Epoch 5/50
5145/5145 [==============================] - 3s 595us/step - loss: 0.5024 - accuracy: 0.7884 - val_loss: 0.4798 - val_accuracy: 0.7864
Epoch 6/50
5145/5145 [==============================] - 3s 596us/step - loss: 0.4710 - accuracy: 0.7892 - val_loss: 0.4786 - val_accuracy: 0.7851
Epoch 7/50
5145/5145 [==============================] - 3s 589us/step - loss: 0.4698 - accuracy: 0.7908 - val_l

In [56]:
with tf.device('/cpu:0'):
    simple_model.evaluate(x=X_test, y=Y_test)

1589/1589 [==============================] - 1s 391us/step - loss: 0.4279 - accuracy: 0.8149


In [57]:
evaluate_per_pid(df, range(50, 100), simple_model, X_test, name="simple_nn_tsfresh")

1589/1589 [==============================] - 1s 360us/step


,Accuracy,MCC,F1_weighted
mean,0.815,0.553,0.815
std,0.067,0.136,0.068


#### CNN + LSTM Model

In [58]:
def cnn_lstm_model(cnn_d = 32, lstm_d = 16):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(cnn_d, kernel_size=(5,), padding='same'))
    
    # Batch Normalization was resulting into NAN due to vanishing coefficients
    # model.add(tf.keras.layers.BatchNormalization(epsilon=1e-04, axis=-1, momentum=0.9))
    model.add(tf.keras.layers.Activation(tf.nn.relu))
    
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.LSTM(lstm_d, return_sequences=False))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid", name='output'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

cnnlstm_model = cnn_lstm_model()

In [59]:
reset_seed()

# Align col has to be set to True to run this model
with tf.device('/cpu:0'):
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    history = cnnlstm_model.fit(X_train, Y_train, 
                               validation_data=(X_val, Y_val), 
                               epochs=50, 
                               shuffle=True,
                               batch_size=8,
                               callbacks=[early_stop_callback])

Epoch 1/50
5145/5145 [==============================] - 8s 1ms/step - loss: 0.5776 - accuracy: 0.7295 - val_loss: 0.5563 - val_accuracy: 0.7471
Epoch 2/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.5571 - accuracy: 0.7475 - val_loss: 0.5494 - val_accuracy: 0.7517
Epoch 3/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.5559 - accuracy: 0.7509 - val_loss: 0.5469 - val_accuracy: 0.7526
Epoch 4/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.5512 - accuracy: 0.7541 - val_loss: 0.5438 - val_accuracy: 0.7595
Epoch 5/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.5490 - accuracy: 0.7569 - val_loss: 0.5445 - val_accuracy: 0.7569
Epoch 6/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.5441 - accuracy: 0.7501 - val_loss: 0.5307 - val_accuracy: 0.7543
Epoch 7/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.5352 - accuracy: 0.7586 - val_loss: 0.5219 - val_accuracy:

In [62]:
with tf.device('/cpu:0'):
    cnnlstm_model.evaluate(x=X_test, y=Y_test)

1589/1589 [==============================] - 1s 521us/step - loss: 0.4771 - accuracy: 0.8166


In [63]:
evaluate_per_pid(df, range(50, 100), cnnlstm_model, X_test, name="cnnlstm_model_tsfresh")

1589/1589 [==============================] - 1s 479us/step


,Accuracy,MCC,F1_weighted
mean,0.819,0.525,0.802
std,0.067,0.127,0.074


# What if we perform some feature normalization

In [64]:
from sklearn.preprocessing import MinMaxScaler

print(X_train.shape)
print(X_train.min().round(5), X_train.max().round(5)) # -20, 100

scaler = MinMaxScaler(feature_range=(-1,1))
X_train_norm = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
X_val_norm = scaler.transform(X_val.reshape(X_val.shape[0], -1)).reshape(X_val.shape)
X_test_norm = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)

print(X_train.shape)
print(X_train_norm.min().round(5), X_train_norm.max().round(5)) # -1, 1
print(X_val_norm.min().round(5), X_val_norm.max().round(5)) # -1, 1
print(X_test_norm.min().round(5), X_test_norm.max().round(5)) # -1, 1c


(41153, 2, 768)
-2.2203420156016817e+20 419917189998739.2
(41153, 2, 768)
-1.0 1.0
-3.70245 7.7438
-107.42857 7.9027


In [65]:
reset_seed()

# Align_cols = True
with tf.device('/cpu:0'):
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    history = simple_model.fit(X_train_norm, Y_train,
                               validation_data=(X_val_norm, Y_val), 
                               epochs=50, 
                               batch_size=8,
                               shuffle=True,
                               callbacks=[early_stop_callback])


Epoch 1/50
5145/5145 [==============================] - 3s 618us/step - loss: 0.4601 - accuracy: 0.8026 - val_loss: 0.4385 - val_accuracy: 0.8158
Epoch 2/50
5145/5145 [==============================] - 3s 593us/step - loss: 0.4392 - accuracy: 0.8126 - val_loss: 0.4501 - val_accuracy: 0.8119
Epoch 3/50
5145/5145 [==============================] - 3s 585us/step - loss: 0.4361 - accuracy: 0.8142 - val_loss: 0.4295 - val_accuracy: 0.8167
Epoch 4/50
5145/5145 [==============================] - 3s 595us/step - loss: 0.4337 - accuracy: 0.8153 - val_loss: 0.4259 - val_accuracy: 0.8193
Epoch 5/50
5145/5145 [==============================] - 3s 594us/step - loss: 0.4304 - accuracy: 0.8166 - val_loss: 0.4262 - val_accuracy: 0.8184
Epoch 6/50
5145/5145 [==============================] - 3s 602us/step - loss: 0.4303 - accuracy: 0.8156 - val_loss: 0.4302 - val_accuracy: 0.8138
Epoch 7/50
5145/5145 [==============================] - 3s 588us/step - loss: 0.4290 - accuracy: 0.8163 - val_loss: 0.4252 -

In [66]:
with tf.device('/cpu:0'):
    simple_model.evaluate(x=X_test_norm, y=Y_test)

1589/1589 [==============================] - 1s 394us/step - loss: 0.3749 - accuracy: 0.8481


In [67]:
with tf.device('/cpu:0'):
    p = simple_model.predict(X_test_norm)
    
print(p.min(), p.max())

1589/1589 [==============================] - 1s 352us/step
1.774295e-21 0.9966631


In [68]:
evaluate_per_pid(df, range(50, 100), simple_model, X_test_norm, name="simple_nn_tsfresh_norm")

1589/1589 [==============================] - 1s 355us/step


,Accuracy,MCC,F1_weighted
mean,0.849,0.614,0.841
std,0.066,0.141,0.070


In [69]:
reset_seed()
# Align_cols = True
with tf.device('/cpu:0'):
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    history = cnnlstm_model.fit(X_train_norm, Y_train,
                               validation_data=(X_val_norm, Y_val), 
                               epochs=50, 
                               batch_size=8,
                               shuffle=True,
                               callbacks=[early_stop_callback])


Epoch 1/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.5476 - accuracy: 0.7426 - val_loss: 0.4602 - val_accuracy: 0.7800
Epoch 2/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.4725 - accuracy: 0.7912 - val_loss: 0.4398 - val_accuracy: 0.8182
Epoch 3/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.4511 - accuracy: 0.8060 - val_loss: 0.4412 - val_accuracy: 0.8147
Epoch 4/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.4428 - accuracy: 0.8088 - val_loss: 0.4193 - val_accuracy: 0.8202
Epoch 5/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.4392 - accuracy: 0.8100 - val_loss: 0.4185 - val_accuracy: 0.8203
Epoch 6/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.4377 - accuracy: 0.8106 - val_loss: 0.4328 - val_accuracy: 0.8143
Epoch 7/50
5145/5145 [==============================] - 7s 1ms/step - loss: 0.4372 - accuracy: 0.8118 - val_loss: 0.4245 - val_accuracy:

In [70]:
with tf.device('/cpu:0'):
    cnnlstm_model.evaluate(x=X_test_norm, y=Y_test)

1589/1589 [==============================] - 1s 514us/step - loss: 0.3763 - accuracy: 0.8481


In [71]:
with tf.device('/cpu:0'):
    p = cnnlstm_model.predict(X_test_norm)
    
print(p.min(), p.max())

1589/1589 [==============================] - 1s 525us/step
0.030230632 0.89968103


In [72]:
evaluate_per_pid(df, range(50, 100), cnnlstm_model, X_test_norm, name="cnnlstm_model_tsfresh_norm")

1589/1589 [==============================] - 1s 488us/step


,Accuracy,MCC,F1_weighted
mean,0.849,0.615,0.842
std,0.066,0.143,0.070
